In [1]:
import random, os, tqdm, time, json
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../../")

random.seed(618)
np.random.seed(907)

new_base_path = os.path.join(
    "/Users/minkexiu/Downloads/",
    "/".join(
        os.getcwd().split("/")[-1*(len(sys.path[-1].split("/")) - 1):]
    ),
)
print("storage dir:", new_base_path)
print("code dir:", os.getcwd())

## 创建文件夹。
if not os.path.exists(new_base_path):
    os.makedirs(
        new_base_path
    )
if not os.path.exists(os.path.join(new_base_path, "preprocessedData")):
    os.makedirs(
        os.path.join(new_base_path, "preprocessedData")
    )
if not os.path.exists(os.path.join(new_base_path, "originalData")):
    os.makedirs(
        os.path.join(new_base_path, "originalData")
    )
if not os.path.exists(os.path.join(new_base_path, "trained_models")):
    os.makedirs(
        os.path.join(new_base_path, "trained_models")
    )

def create_originalData_path(filename_or_path):
    return os.path.join(new_base_path, "originalData", filename_or_path)
def create_preprocessedData_path(filename_or_path):
    return os.path.join(new_base_path, "preprocessedData", filename_or_path)
def create_trained_models_path(filename_or_path):
    return os.path.join(new_base_path, "trained_models", filename_or_path)

def millisec2datetime(timestamp):
    time_local = time.localtime(timestamp/1000)
    return time.strftime("%Y-%m-%d %H:%M:%S", time_local)
    
def run_finish():
    # 假设你的字体文件是 'myfont.ttf' 并且位于当前目录下  
    font = FontProperties(fname="/Users/minkexiu/Documents/GitHub/ML_Tryout/SimHei.ttf", size=24)  
    # 创建一个空白的图形  
    fig, ax = plt.subplots()  
    ax.imshow(
        plt.imread("/Users/minkexiu/Downloads/wallhaven-dgxpyg.jpg")
    )
    # 在图形中添加文字  
    ax.text(
        ax.get_xlim()[1] * 0.5, 
        ax.get_ylim()[0] * 0.5, 
        f"程序于这个点跑完：\n{millisec2datetime(time.time()*1000)}", fontproperties=font, ha="center", va="center", color="red"
    )  
    # 设置图形的布局  
    # ax.set_xlim(0, 1)  
    # ax.set_ylim(0, 1)  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.patch.set_color("blue")
    # 显示图形  
    plt.show()
        
tqdm.tqdm.pandas() ## 引入这个，就可以在apply的时候用progress_apply了。

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True) 
    
def simply_show_data(df1):
    print(df1.shape)
    display(df1.head())
    
def wait_flag(saved_flag_path, time_interval_sec=10):
    print("waiting for", saved_flag_path)
    time_count = 0
    while True:
        if os.path.exists(saved_flag_path):
            break
        time.sleep(time_interval_sec)
        time_count+=time_interval_sec
        print(time_count, end=" ")
    print("finish!!")

storage dir: /Users/minkexiu/Downloads/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO
code dir: /Users/minkexiu/Documents/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO


# (之前的草稿)

In [2]:
## 加载acm的tokenizer，
## 加载一下数据试试看。

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(create_trained_models_path("Qwen1.5-0.5B-Chat"))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
tokenizer

Qwen2TokenizerFast(name_or_path='/Users/minkexiu/Downloads/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO/trained_models/Qwen1.5-0.5B-Chat', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
## 加载一下数据：
with open(
    "/Users/minkexiu/Documents/GitHub/ML_runCodeFromBook/大规模语言模型：从理论到实践/RLHF_PPO/data/train_data.json", 
    'r', 
    encoding="utf-8"
) as f:
    data = json.load(f)

In [9]:
data
## 数据大概长这样。有一个query，有一个system_content。

[{'query': '饭店服务员的态度太差，使用委婉积极的态度投诉', 'system_content': '你是一个有文化的文明人'},
 {'query': '领导故意刁难你，你想骂他娘的，使用文明语言骂他娘的', 'system_content': '你是一个有文化的文明人'}]

# 尝试一下呗。

In [4]:
import torch

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
from peft import LoraConfig, get_peft_model, PeftModel

# 初始化一个演员·评论员模型

## 准备数据

In [ ]:
ckpt

## 演员·评论员

In [18]:
## 尝试初始化一下原始的文本生成模型.
## 这个模型是一个生成模型哦。
model = AutoModelForCausalLM.from_pretrained(
    create_trained_models_path("Qwen1.5-0.5B-Chat")
).to("cpu").eval()

## Lora

In [11]:
from dataclasses import dataclass, field

In [16]:
lora_config = LoraConfig(
    r=2, ## 把秩降到这个数。
    lora_alpha=8, ## 这个是一个扩张系数。
    target_modules=['k_proj',  'v_proj'],
    lora_dropout=0,
    task_type="CAUSAL_LM",
)

In [22]:
lora_model = PeftModel(model, lora_config)
v_head = torch.nn.Linear(1024, 1, bias=False).to("cpu")
for name, module in lora_model.named_modules():
    if 'lora_' in name:
        for param in module.parameters():
            param.requires_grad = True

In [ ]:
def lora_forward(input_ids, attention_mask, tools):
    res = lora_model.forward(input_ids, attention_mask, output_hidden_states=True)
    values = v_head(res.hidden_states[0]).squeeze(-1)[:, :-1]